<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/spacy_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
ls

images/  pending_images/  reviews_text.csv


In [0]:
import pandas as pd
import numpy as np
import random
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
from spacy import displacy
from spacy.matcher import Matcher
# Import the English language class
from spacy.lang.en import English

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [44]:
!pip install spacymoji
from spacymoji import Emoji

     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=e7e2ad5967a844e71573e57df2cf7acddbe20fcfaa1b14829b1f04a9965bfc3f
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
nlp = spacy.load('en_core_web_lg', disable = ['ner'])
matcher = Matcher(nlp.vocab)

In [7]:
df_review = pd.read_csv('reviews_text.csv')
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 36 columns):
Unnamed: 0           56156 non-null int64
_id                  56156 non-null object
updatedAt            56156 non-null object
createdAt            56156 non-null object
originality          56156 non-null int64
value                56156 non-null int64
nutrition            56156 non-null int64
presentation         56156 non-null int64
taste                56156 non-null int64
text                 56155 non-null object
dish                 56156 non-null object
type                 56156 non-null object
isPublished          5885 non-null object
user                 56156 non-null object
restaurant           56156 non-null object
images               56156 non-null object
likes                56156 non-null object
__v                  56156 non-null int64
comments             56156 non-null object
likesCount           56156 non-null int64
tags                 56156 non-null obje

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,25,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
review_text = df_review[['_id','text']]
review_text.head()

,_id,text
0,59e599911747c90004ba8586,Burger joint offers a wide range of cheeseburg...
1,59ff122b8b04fd0004df6c78,It was really good. The mushroom broth was esp...
2,5dc251e552e7e90020b6adfb,"$8.90 for sesame rice, mushroom rendang, curry..."
3,5d35cf4049745d0004c68de3,2 mains + 1 green rice bento from greendot. Re...
4,5d304e9264f5e5000423b75b,The lion mane mushroom rendang is so delicious...


In [0]:
# Combine the # and tag to form hashtag

matcher.add('HASHTAG', None, [{'ORTH': '#'}, {'IS_ASCII': True}])

doc = nlp('This is a #sentence. Here is another #hashtag. #The #End.')
matches = matcher(doc)
hashtags = []
for match_id, start, end in matches:
    hashtags.append(doc[start:end])

for span in hashtags:
    span.merge()

print([t.text for t in doc])

In [11]:
# Select a random review from the corpus
check = random.randint(0,20000)
review_sample = review_text['text'][check:check+5]
#print(review_sample)

# convert that into nlp
df_token = pd.DataFrame()

stop_word_list = []
punctuation_list = []
non_english_list = []
det_list = []
adj_list = []
noun_list = []
adverb_list = []
verb_list = []
other_pos_list = []
prop_noun_list = []
hashtags = []

for sample in review_sample:
  review_doc = nlp(sample)

# Break those into sentences
  print('----')
  for sentence in review_doc.sents:
    print(sentence.text)

  matches = matcher(review_doc)

  for match_id, start, end in matches:
    hashtags.append(review_doc[start:end])

  for span in hashtags:
    span.merge()


# Find the part of speech
  print('----')

  tokens_list = []
  for token in review_doc:
    token_dict = {}
    token_dict['text'] = token.text
    token_dict['lemma'] = token.lemma_
    token_dict['part_of_speech'] = token.pos_
    token_dict['dependency'] = token.dep_
    token_dict['shape'] = token.shape_
    token_dict['alphabet?'] = token.is_alpha
    token_dict['stop_word'] = token.is_stop
    token_dict['punctuation'] = token.is_punct
    token_dict['digit'] = token.is_digit
    token_dict['language'] = token.lang_
    token_dict['prefix'] = token.prefix_
    token_dict['sentiment'] = token.sentiment
    token_dict['currency'] = token.is_currency

    tokens_list.append(token_dict)
 
    if token.is_stop:
      stop_word_list.append(token)
    if token.is_punct:
      punctuation_list.append(token)
    if token.lang_ != 'en':
      non_english_list.append(token)
    if token.pos_ == 'DET':
      det_list.append(token)
    elif token.pos_ == 'ADJ':
      adj_list.append(token.lemma_)
    elif token.pos_ == 'NOUN':
      noun_list.append(token.lemma_)
    elif token.pos_ == 'ADV':
      adverb_list.append(token.lemma_)
    elif token.pos_ == 'VERB':
      verb_list.append(token.lemma_)
    elif token.pos_ == 'PROPN':
      prop_noun_list.append(token.lemma_)
    else:
      other_pos_list.append(token)

    df_token = pd.concat([df_token, pd.DataFrame(tokens_list)],axis=0)

print("Noun List: {}".format(set(noun_list)))
print("Proper Noun List: {}".format(set(prop_noun_list)))
print("Adjective List: {}".format(set(adj_list)))
print("Verb List: {}".format(set(verb_list)))
print("Adverb List: {}".format(set(adverb_list)))
print("Det List: {}".format(set(det_list)))
print("Other POS List: {}".format(set(other_pos_list)))
print("Non-English: {}".format(set(non_english_list)))
print("Punctuations: {}".format(set(punctuation_list)))
print("Stop Words: {}".format(set(stop_word_list)))

df_token

----
One of the best if not the best green mango salad we have had.
Just the right amount of spice and really fresh and well made.
----
----
We expected this to follow the same standards as other dishes but this dish could be a lot better.
Not too spicy and just like any other place this dish was so so
----
----
This Korean Mexican fushion restaurant in Central is a tiny place to grab a cheap lunch.
You can order the eggplant filling as a burrito, tacos or salad.
It’s vegetarian but if you omit the sour cream and cheese, it’s vegan.
The tacos are pretty big and come with large chunks of eggplant, chillies, fresh veggies, sesame seeds, and ginger sauce.
----
----
This cafe is part of a guest house in Chiang Mai.
It has tons of vegan and veggie options, including Thai and Western food.
This curry was fresh, delicious, and not too spicy.
Order with brown rice!
----
----
Pennywort is apparently a superfood in Asia, which I’d never tried before.
This salad was fresh and delicious.
The place

,text,lemma,part_of_speech,dependency,shape,alphabet?,stop_word,punctuation,digit,language,prefix,sentiment,currency
0,One,one,NUM,ROOT,Xxx,True,True,False,False,en,O,0.0,False
0,One,one,NUM,ROOT,Xxx,True,True,False,False,en,O,0.0,False
1,of,of,ADP,prep,xx,True,True,False,False,en,o,0.0,False
0,One,one,NUM,ROOT,Xxx,True,True,False,False,en,O,0.0,False
1,of,of,ADP,prep,xx,True,True,False,False,en,o,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,towards,towards,ADP,prep,xxxx,True,True,False,False,en,t,0.0,False
76,the,the,DET,det,xxx,True,True,False,False,en,t,0.0,False
77,learning,learning,NOUN,compound,xxxx,True,False,False,False,en,l,0.0,False
78,centre,centre,NOUN,pobj,xxxx,True,False,False,False,en,c,0.0,False


In [12]:
hashtags



[]

In [0]:
similarity_list = []
for token1 in review_doc:
  for token2 in review_doc:
    sim_dict = {}
    sim_dict['word_1'] = token1.text
    sim_dict['word_2'] = token2.text

    try:
      sim_dict['similarity'] = token1.similarity(token2)
    except:
      print('error')


    similarity_list.append(sim_dict)

df_similarity = pd.DataFrame(similarity_list)\
.query('similarity < 0.9999')\
.sort_values(['similarity'],ascending=False)
df_similarity

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty v

,word_1,word_2,similarity
194,broc,cauli,0.649460
222,cauli,broc,0.649460
198,and,for,0.570359
58,for,and,0.570359
5,love,!,0.450742
...,...,...,...
177,x,broc,-0.147135
213,cauli,for,-0.182374
59,for,cauli,-0.182374
183,broc,for,-0.194480


In [41]:
displacy.render(nlp("Seitan hot dog was really good and flavourful with a good bounce that you want in a hot dog."),'dep',True,0,jupyter=True)

In [0]:


# Write a pattern for Vegan Chicken
pattern_vegan_chicken = [{"TEXT": "vegan"}, {"TEXT": "chicken"}]

# Write a pattern for ice cream
pattern_ice_cream = [{"TEXT": "ice"}, {"TEXT": "cream"}]
pattern_laksa = [{"TEXT": "laksa"}]
pattern_pizza = [{"TEXT": "pizza"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add('HASHTAG', None, [{'ORTH': '#'}, {'IS_ASCII': True}])
matcher.add('hyphenated',None,[{'IS_ASCII': True},{'ORTH': '-'},{'IS_ASCII': True}])
matcher.add("propn_adj_noun", None, [{'POS': 'PROPN'}, {'POS': 'ADJ'}, {'POS': 'NOUN'}])
matcher.add("adj_noun", None, [{'POS': 'NOUN'}, {'POS': 'NOUN'}])
#matcher.add("vegan_chicken", None, pattern_vegan_chicken)
#matcher.add("ice_cream", None, pattern_ice_cream)
#matcher.add("laksa", None, pattern_laksa)
#matcher.add("pizza", None, pattern_pizza)

In [33]:
sentence1 = "Seitan hot dog was really good and flavourful with a good bounce that you want in a hot dog."
sentence2 = "This burger is one of the reasons why non-vegans say vegan food tastes bad."

sent_list = [sentence1, sentence2]

for sentence in sent_list:
  matches = matcher(nlp(sentence))
  print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
  for match_id, start, end in matches:
    print("Match found:", nlp(sentence)[start:end].text)

Total matches found: 1
Match found: Seitan hot dog
Total matches found: 2
Match found: non-
Match found: non-vegans


In [0]:
doc = nlp("A phrase with another phrase occurs.")
len(doc.noun_chunks_)


AttributeError: ignored

In [0]:
list(nlp(sentence1).noun_chunks)
list(nlp(sentence2).noun_chunks)

[Seitan hot dog, a good bounce, you, a hot dog]

[Vegan risotto]

In [0]:
assert chunks[0].text == "A phrase"
assert chunks[1].text == "another phrase"

In [0]:
review_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 2 columns):
_id     56156 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 877.6+ KB


In [45]:
# Select a random review from the corpus
check = random.randint(0,20000)
review_sample = review_text['text'][check:check+100]
#print(review_sample)



matches_list = []

for sample in review_sample:
  review_doc = nlp(sample)
  matches = matcher(review_doc)
  print(len(matches), sample )

# Iterate over the matches and print the span text
  for match_id, start, end in matches:
    matches_list.append(review_doc[start:end].text)


matches_list

0 One of the best mooncakes I’ve tried so far :3 Nice taste of green tea & red bean although it’s on the sweeter side!
4 Fresh red bean flavour and not too sweet. Classic combination done well. #sgeats #mooncake #redbean
3 My favourite #mooncake in Singapore mainly because of the textures, umami and perfect balance between savoury and sweet. Tip: cut with a sharp knife not the butter knife 😂 #bestpicks
4 My favourite locally made mooncake. The yolk is made of carrots I believe. It doesn’t taste like carrots at all. Very umami, has the texture of egg yolk and well salted. Contrasts well with the sweet lotus paste. #sgeats #mooncake
2 Soy fanática de los Hor fun en general, pero estos no me gustaron mucho porque le pusieron unos ajíes que venían en vinagre y como que no combinaban con el sabor del plato😟
0 The green chillies in this were quite essential. While it looks starchy, it's not too oily. Quite alright for a $4.50 lunch.
0 I really enjoyed my Hor Fun. I added some extra “mock mea

['bean flavour',
 '#sgeats',
 '#mooncake',
 '#redbean',
 '#mooncake',
 'butter knife',
 '#bestpicks',
 'egg yolk',
 'lotus paste',
 '#sgeats',
 '#mooncake',
 'Soy fanática',
 'unos ajíes',
 'hor fun',
 '#neveragain',
 '#sgeats',
 '#horfun',
 'size portions',
 'bbq sauce',
 'serve fix',
 'tomato sauce',
 '#vivavanderbilt',
 'Sesame ginger',
 'vanilla bean',
 'bean pastry',
 'pastry cream',
 'chocolate icing',
 'Vanilla bean',
 'bean cream',
 'chocolate frosting',
 'raspberry curd',
 'curd filling',
 'raspberry curd',
 'vanilla icing',
 'apple fritter',
 'maple frosting',
 'comfort food',
 'Chick-un',
 '#comfortfood',
 '#bostonvegan',
 'smoothie bowls',
 'maqui berry',
 'açaí berry',
 'summer berries',
 'almond butter',
 'housemade granola',
 'granola - and',
 'breakfast option',
 '#smoothiebowl',
 'smoothiebowl #',
 '#pressed',
 '#veganboston',
 'ice cream',
 'cream push',
 'push pops',
 '#dessert',
 '#dessert',
 '#bychloe',
 'cashew crema',
 'crema sauce',
 '#nachos',
 '#veganamsterdam

In [17]:
noun_chunks_list

[[It,
  It,
  aubergine and pepper pieces,
  the aubergine,
  The dish,
  a party,
  flavours],
 [a classic açaí bowl,
  different fruit,
  chia seeds,
  coconut flakes,
  granola,
  açaí,
  quite a large portion,
  you,
  the staff,
  you,
  they,
  you,
  the bowl,
  plastic containers 🤗],
 [the açaí base,
  the chia,
  a bit more fruit,
  top,
  it,
  the best açaí bowl,
  i,
  singapore,
  they,
  plastic cutlery,
  bowls,
  i,
  it,
  the shop],
 [Their acai, something, it, the granola],
 [granola,
  chia seed,
  blueberries,
  banana,
  coconut flakes,
  they,
  it,
  cacao nibs 🙆‍♂]]